In [41]:
import os
import sys
import copy
from typing import List, Tuple, Dict
from sage.graphs.connectivity import connected_components

Util

In [42]:
def generate_transposition(n: int, u: int, v: int) -> Permutation:
    action = list(range(1, n + 1))
    action[u - 1] = v
    action[v - 1] = u
    return Permutation(action)

generate_transposition(4, 2, 4)  # [1, 4, 3, 2]

[1, 4, 3, 2]

Input verification

In [43]:
def validate_edges(n: int, allowed_pairs: List[Tuple[int, int]]):
    """Checks for invalid edges, duplicate edges (up to ordering), and self-loops."""
    elements = [u for pair in allowed_pairs for u in pair]
    if not (1 <= min(elements) and max(elements) <= n):
        print("Element out of range")
        return False

    for (u, v) in allowed_pairs:
        if u == v:
            print(f"Self-loop found: {(u, v)}")
            return False

    pairs = set([(min(u, v), max(u, v)) for (u, v) in allowed_pairs])
    if len(pairs) != len(allowed_pairs):
        print("Duplicate edge found.")
        return False

    if not Graph([list(range(1, n + 1)), allowed_pairs]).is_connected():
        print("Graph is not connected")
        return False

    return True

CC = connected component

`dp[rank of permutation, CC's] = (min length, number of factorizations with such length)`

In [53]:
Edge = Tuple[int, int]

def merge_cc(cc_list, u: int, v: int):
    cc_u = [cc for cc in cc_list if u in cc]
    cc_v = [cc for cc in cc_list if v in cc]
    assert len(cc_u) == 1 and len(cc_v) == 1
    cc_u = cc_u[0]
    cc_v = cc_v[0]
    
    if cc_u == cc_v:
        return list(map(list, cc_list))
    new_cc_list = list(cc_list)
    new_cc_list.remove(cc_u)
    new_cc_list.remove(cc_v)
    new_cc = sorted(cc_u + cc_v)
    new_cc_list.append(new_cc)
    return new_cc_list


def compute_dp(n: int,
               allowed_edges: List[Tuple[int, int]],
               dp: Dict[int, List[List[int]]],
               rank: int,
               cc_list: List[List[int]]) -> Tuple[int, int]:
    cc_list = tuple(map(tuple, cc_list))
    if (rank, cc_list) in dp:
        return dp[(rank, cc_list)]

    if rank == 0 and len(cc_list) == 1:
        dp[(rank, cc_list)] = (0, 1)
        return dp[(rank, cc_list)]

    dp[(rank, cc_list)] = None
    perm = Permutations(n).unrank(rank)

    cur_min_length = -1
    cur_count = 0

    # Try all transpositions
    for (u, v) in allowed_edges:
        # Get new rank
        action = generate_transposition(n, u, v)
        new_perm = Permutation(perm).right_action_product(action)
        new_rank = Permutations(n).rank(new_perm)

        # Get new cc list
        new_cc_list = merge_cc(cc_list, u, v)

        # Update DP
        next_dp = compute_dp(n, allowed_edges, dp, new_rank, new_cc_list)
        if next_dp is None:
            continue
        if cur_min_length == -1 or next_dp[0] < cur_min_length:
            cur_min_length, cur_count = next_dp
        elif next_dp[0] == cur_min_length:
            cur_count += next_dp[1]

    dp[(rank, cc_list)] = (cur_min_length + 1, cur_count)
    return dp[(rank, cc_list)]

`compute_length_n(n)`: Returns a map from permutation rank to (min length of factorization, number of such factorizations). Keys are ranks of permutations of length n.

In [54]:
def compute_length_n(n: int, allowed_edges: List[Tuple[int, int]]):
    starting_cc_list = [[i] for i in range(1, n + 1)]
    starting_cc_tuple = tuple(map(tuple, starting_cc_list))

    dp = {}
    perm_n = Permutations(n)
    for rank in range(factorial(n)):
        perm = perm_n.unrank(rank)
        compute_dp(n, allowed_edges, dp, rank, starting_cc_list)
        min_length, count = dp[(rank, starting_cc_tuple)]
        print(f"{rank}: {perm}: min_len = {min_length}, count = {count}")
    result = {rank: dp[(rank, starting_cc_tuple)] for rank in range(factorial(n))}
    return result

In [55]:
def main(n: int, allowed_edges: List[Tuple[int, int]]):
    if not validate_edges(n, allowed_edges):
        return
    results = compute_length_n(n, allowed_edges)
    return results

In [62]:
n = 4
test_allowed_pairs = [(1, 2), (1, 3), (1, 4)]
main(n, test_allowed_pairs)

Duplicate edge found.
